In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
# Enable imports from src
import sys
PROJ_ROOT = os.path.abspath('../')
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
import glob
import h5py
import tqdm
import misc

%matplotlib inline

In [2]:
data_path = os.path.join(PROJ_ROOT, 'data/raw/Ultrasound_high_fps/DL2019Bmode/*/*.h5')
h5files = glob.glob(data_path)
len(h5files)

126

In [9]:
data = h5py.File(h5files[6])

In [10]:
data['tissue/data'].shape

(320, 320, 168)

In [15]:
video = np.transpose(data['tissue/data'], [2, 1, 0])
fps = 1 / (data['tissue/times'][5] - data['tissue/times'][4])

In [16]:
misc.ultraSoundAnimation(video, fps=fps)

In [26]:
data.close()

In [18]:
enhanced = 1 / (1 + np.exp(-((video / 255.) - 0.6) * .7))
misc.ultraSoundAnimation(enhanced, fps=fps)

In [19]:
pal_txt = open('../../pal.txt')
line = pal_txt.readline()[:-1]
pal = np.array([int(val) for val in line.split(',')])

In [24]:
pal_enhanced = np.array([[[pal[int(video[i, j, k])] for k in range(video.shape[2])] for j in range(video.shape[1])] for i in range(video.shape[0])])

In [25]:
misc.ultraSoundAnimation(pal_enhanced, fps=fps)

In [3]:
shapes = []
for h5file in h5files:
    with h5py.File(h5file) as data:
        shapes.append(data['tissue/data'].shape)

In [4]:
max_len = max(shapes, key=lambda x: x[2])[2]
max_height = max(shapes, key=lambda x: x[1])[1]
max_width = max(shapes, key=lambda x: x[0])[0]
print(max_len, max_height, max_width)

236 380 537


In [5]:
sum([x[2] for x in shapes])

15699

In [6]:
len(shapes)

126

In [7]:
i = 0
for h5file in h5files:
    with h5py.File(h5file) as data:
        print(i)
        i += data['tissue/data'].shape[2]

0
92
187
284
457
623
792
960
1137
1318
1488
1649
1814
1933
2047
2163
2312
2442
2575
2716
2852
2990
3099
3203
3305
3438
3564
3742
3934
4100
4336
4439
4558
4682
4795
4909
5024
5140
5255
5377
5478
5609
5757
5921
6081
6214
6321
6443
6542
6600
6658
6716
6881
7038
7201
7305
7411
7521
7646
7785
7919
8015
8115
8215
8324
8428
8538
8620
8701
8794
8877
8954
9019
9117
9199
9281
9411
9540
9650
9785
9893
10004
10087
10198
10268
10399
10526
10652
10799
10920
11050
11155
11262
11369
11494
11615
11750
11946
12103
12267
12451
12633
12782
12905
13020
13137
13222
13306
13401
13533
13664
13797
13920
14046
14162
14289
14411
14550
14737
14924
15114
15216
15318
15425
15529
15610


In [8]:
h5_idc = list(range(0, len(h5files)))
np.random.shuffle(h5_idc)

In [9]:
train_num = int(round(0.8 * len(h5files)))

In [10]:
train_num

101

In [11]:
train_idc = h5_idc[:train_num]
val_idc = h5_idc[train_num:]

In [12]:
max_train_width = max([shapes[i][0] for i in train_idc])
max_train_height = max([shapes[i][1] for i in train_idc])
num_train_frames = sum([(shapes[i][2] - 1) for i in train_idc])

In [13]:
num_train_frames

12479

In [14]:
train_path = os.path.join(PROJ_ROOT, 'data/processed/train_low_fps.h5')
train_data = h5py.File(train_path, 'w')

In [15]:
train_data.create_dataset('fixed', shape=(num_train_frames, max_train_height, max_train_width, 1))
train_data.create_dataset('moving', shape=(num_train_frames, max_train_height, max_train_width, 1))

<HDF5 dataset "moving": shape (12479, 380, 537, 1), type "<f4">

In [45]:
train_data['fixed'][3:5].shape

(2, 380, 537, 1)

In [16]:
frame_idx = 0

for i in tqdm.tqdm_notebook(train_idc):
    with h5py.File(h5files[i]) as data:
        video = np.transpose(data['tissue/data'], [2, 1, 0])
        height = video.shape[1]
        width = video.shape[2]
        num_frames = video.shape[0]
        # video = np.array([[[float(pal[int(video[i, j, k])])
        #                            for k in range(video.shape[2])] 
        #                           for j in range(video.shape[1])] 
        #                          for i in range(video.shape[0])])
        h_diff = max_train_height - height
        w_diff = max_train_width - width
        if h_diff % 2 == 0:
            h_add_one = 0
        else:
            h_add_one = 1
        if w_diff % 2 == 0:
            w_add_one = 0
        else:
            w_add_one = 1
            
        video = np.pad(video, [[0, 0], 
                               [h_diff // 2, h_diff // 2 + h_add_one], 
                               [w_diff // 2, w_diff // 2 + w_add_one]],
                      mode='constant')
        video /= 255.
        
        fixed = video[:-1, :, :, None]
        moving = video[1:, :, :, None]
        # print(frame_idx, num_frames, fixed.shape)
        
        train_data['fixed'][frame_idx:frame_idx + num_frames - 1] = fixed
        print(train_data['fixed'][frame_idx:frame_idx + num_frames - 1].shape[0], fixed.shape[0],
             train_data['moving'][frame_idx:frame_idx + num_frames - 1].shape[0], moving.shape[0])
        train_data['moving'][frame_idx:frame_idx + num_frames - 1] = moving
        frame_idx += (num_frames - 1)

106 106 106 106
165 165 165 165
91 91 91 91
183 183 183 183
181 181 181 181
132 132 132 132
167 167 167 167
82 82 82 82
177 177 177 177
164 164 164 164
156 156 156 156
57 57 57 57
120 120 120 120
133 133 133 133
132 132 132 132
110 110 110 110
115 115 115 115
168 168 168 168
103 103 103 103
176 176 176 176
164 164 164 164
98 98 98 98
64 64 64 64
83 83 83 83
138 138 138 138
106 106 106 106
94 94 94 94
113 113 113 113
121 121 121 121
97 97 97 97
103 103 103 103
163 163 163 163
114 114 114 114
112 112 112 112
159 159 159 159
129 129 129 129
104 104 104 104
84 84 84 84
103 103 103 103
235 235 235 235
129 129 129 129
122 122 122 122
116 116 116 116
109 109 109 109
115 115 115 115
92 92 92 92
81 81 81 81
132 132 132 132
186 186 186 186
106 106 106 106
148 148 148 148
99 99 99 99
109 109 109 109
130 130 130 130
126 126 126 126
103 103 103 103
148 148 148 148
132 132 132 132
186 186 186 186
125 125 125 125
113 113 113 113
106 106 106 106
134 134 134 134
114 114 114 114
135 135 135 135
76 76 76

In [17]:
train_data.close()

In [18]:
max_val_width = max([shapes[i][0] for i in val_idc])
max_val_height = max([shapes[i][1] for i in val_idc])
num_val_frames = sum([(shapes[i][2] - 1) for i in val_idc])

In [19]:
val_path = os.path.join(PROJ_ROOT, 'data/processed/val_low_fps.h5')
val_data = h5py.File(val_path, 'w')

In [20]:
val_data.create_dataset('fixed', shape=(num_val_frames, max_val_height, max_val_width, 1))
val_data.create_dataset('moving', shape=(num_val_frames, max_val_height, max_val_width, 1))

frame_idx = 0
for i in tqdm.tqdm_notebook(val_idc):
    with h5py.File(h5files[i]) as data:
        video = np.transpose(data['tissue/data'], [2, 1, 0])
        height = video.shape[1]
        width = video.shape[2]
        num_frames = video.shape[0]
        # video = np.array([[[float(pal[int(video[i, j, k])])
        #                             for k in range(video.shape[2])] 
        #                            for j in range(video.shape[1])] 
        #                           for i in range(video.shape[0])])
        h_diff = max_val_height - height
        w_diff = max_val_width - width
        if h_diff % 2 == 0:
            h_add_one = 0
        else:
            h_add_one = 1
        if w_diff % 2 == 0:
            w_add_one = 0
        else:
            w_add_one = 1
            
        video = np.pad(video, [[0, 0], 
                               [h_diff // 2, h_diff // 2 + h_add_one], 
                               [w_diff // 2, w_diff // 2 + w_add_one]],
                      mode='constant')
        video /= 255.
        
        fixed = video[:-1, :, :, None]
        moving = video[1:, :, :, None]
        
        # print(frame_idx, num_frames, fixed.shape)
        
        val_data['fixed'][frame_idx:frame_idx + num_frames - 1] = fixed
        val_data['moving'][frame_idx:frame_idx + num_frames - 1] = moving
        frame_idx += (num_frames - 1)

In [21]:
val_data.close()

In [ ]:
with h5py.File(train_path) as data:
    for i in range(data['fixed'].shape[0]):
        if (data['fixed'][i, :, :, :] == 0.0).all():
            print(i)